This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
import os
import chromadb
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = "API-KEY-HERE"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:
# Importing the necessary library for OpenAI API
import openai
openai.api_base = "https://openai.vocareum.com/v1"

# Define your OpenAI API key 
api_key = "API-KEY-HERE"
openai.api_key = api_key

In [3]:
prompt_template = '''
Create a real estate listing that  should follow the following format:

Neighborhood: (i.e. Green Oaks)
Price: (i.e."$800,000")
Bedrooms: (i.e. 3)
Bathrooms: (i.e. 2)
House Size: (i.e "2,000" sqft)

Description: (i.e. Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.)

Neighborhood Description: (i.e. Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.)
'''

In [4]:
# Function to call the OpenAI GPT-3.5 API
def generate_real_estate_listings(prompt_template):
    try:
        # Calling the OpenAI API with a system message and our prompt in the user message content
        # Use openai.ChatCompletion.create for openai < 1.0
        # openai.chat.completions.create for openai > 1.0
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": "You are a real estate agent. You are writing up real estate listings for potential buyers to view. "
          },
          {
            "role": "user",
            "content": prompt_template
          }
          ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
        # The response is a JSON object containing more information than the generated review. We want to return only the message content
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

# Generating the response from the model
generated_listings = generate_real_estate_listings(prompt_template)

# Printing the output. 
print("Generated Listings:")
print(generated_listings)

Generated Listings:
Neighborhood: Riverfront Estates
Price: $1,200,000
Bedrooms: 4
Bathrooms: 3.5
House Size: 3,500 sqft

Description: Situated in the prestigious Riverfront Estates, this luxurious 4-bedroom, 3.5-bathroom home offers a blend of modern sophistication and timeless elegance. The grand foyer welcomes you into a spacious, light-filled living room with soaring ceilings and a cozy fireplace. The gourmet kitchen features high-end appliances, granite countertops, and a breakfast nook overlooking the beautifully landscaped backyard. The master suite is a true retreat with a spa-like bathroom and a private balcony with stunning views of the river. Additional features include a home office, a media room, and a three-car garage.

Neighborhood Description: Riverfront Estates is an exclusive waterfront community known for its upscale residences and scenic views of the river. Residents can enjoy access to private docks, walking trails, and a neighborhood clubhouse. The neighborhood is

In [5]:
# Create the data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

In [6]:
def save_text_to_file(text, prefix='document'):
    """
    Saves the given text to a uniquely named file in the data directory.
    """
    # Generate a unique filename using timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"{prefix}_{timestamp}.txt"
    
    # Create the full file path
    file_path = os.path.join('data', filename)
    
    # Write the text to the file
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(text)
        return file_path
    except Exception as e:
        print(f"Error writing to file: {e}")
        return None

In [7]:
# Generate and save 10 listings
for i in range(10):
    # Generate a listing
    listing_text = generate_real_estate_listings(prompt_template)
    
    # Save it to a file with a prefix of 'listing'
    file_path = save_text_to_file(listing_text, prefix='listing')
    
    # Print confirmation
    print(f"Generated listing {i+1}/10: {file_path}")

Generated listing 1/10: data/listing_20241106_113112.txt
Generated listing 2/10: data/listing_20241106_113116.txt
Generated listing 3/10: data/listing_20241106_113120.txt
Generated listing 4/10: data/listing_20241106_113124.txt
Generated listing 5/10: data/listing_20241106_113128.txt
Generated listing 6/10: data/listing_20241106_113132.txt
Generated listing 7/10: data/listing_20241106_113136.txt
Generated listing 8/10: data/listing_20241106_113140.txt
Generated listing 9/10: data/listing_20241106_113145.txt
Generated listing 10/10: data/listing_20241106_113150.txt


In [8]:

def load_and_embed_documents():
    """
    Load text documents from a directory and create a vector store using OpenAI embeddings.
    """
    documents = []
    
    # Load each text file in the directory
    for filename in os.listdir("data"):
        if filename.endswith('.txt'):
            file_path = os.path.join("data", filename)
            loader = TextLoader(file_path)
            documents.extend(loader.load())
            print(f"Loaded {filename}")
    
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings()
    
    # Create and persist vector store
    db = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )
    
    return db

In [9]:
loaded_db = load_and_embed_documents()

Loaded listing_20241106_113145.txt
Loaded listing_20241106_113150.txt
Loaded listing_20241106_113136.txt
Loaded listing_20241106_113140.txt
Loaded listing_20241106_113112.txt
Loaded listing_20241106_113116.txt
Loaded listing_20241106_113132.txt
Loaded listing_20241106_113124.txt
Loaded listing_20241106_113120.txt
Loaded listing_20241106_113128.txt


In [10]:
user_questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
user_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

In [11]:

def get_real_estate_recommendations(user_answers):    
    # Initialize components
    embeddings = OpenAIEmbeddings()
    llm = ChatOpenAI(temperature=1.0)
    
    # Load the vector store
    db = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )
    
    # Create simple prompt
    prompt = ChatPromptTemplate.from_template("""
    As a real estate agent, recommend properties based on these customer preferences:

    Size and Layout: {size}
    Top Priorities: {priorities}
    Desired Amenities: {amenities}
    Transportation: {transport}
    Neighborhood Type: {neighborhood}

    Available Properties:
    {listings}

    Please suggest the best matches and explain why they would work for this customer.
    """)
    
    # Combine all preferences for search
    search_query = f"{user_answers[0]} {user_answers[1]} {user_answers[2]} {user_answers[3]} {user_answers[4]}"
    
    # Get relevant listings
    docs = db.similarity_search(search_query, k=3)
    listings = "\n\n".join(doc.page_content for doc in docs)
    
    # Format the prompt
    messages = prompt.format_messages(
        size=user_answers[0],
        priorities=user_answers[1],
        amenities=user_answers[2],
        transport=user_answers[3],
        neighborhood=user_answers[4],
        listings=listings
    )
    
    # Get recommendations
    response = llm.invoke(messages)
    return response.content

In [12]:
get_real_estate_recommendations(user_answers)

"Based on the customer preferences provided, the best match would be the property in Serene Meadows. \n\nThis property meets the criteria of a comfortable three-bedroom house with a spacious kitchen and a cozy living room. Additionally, it offers a bonus fourth bedroom, providing extra space for guests or as a home office. The property also includes a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. \n\nThe neighborhood of Serene Meadows offers a quiet and family-friendly atmosphere, as well as easy access to parks, playgrounds, and walking trails. It is well-connected to local schools, shopping options, and dining establishments, fulfilling the customer's top priorities. \n\nFurthermore, Serene Meadows provides a balance between suburban tranquility and urban amenities, with convenient access to major highways and public transportation options. Overall, this property in Serene Meadows ticks all the boxes for the customer's preferences and offers